In [1]:
# 读取CSV文件并转换为列表
file_path = "G:\\Hangkai\\Forest_edge\\country_stats\\final_data\\forest_edge_country.csv"

with open(file_path, 'r', encoding='ISO-8859-1') as f:
    lines = f.readlines()
    header = lines[0].strip().split(',')
    lines = lines[1:]  # 仅获取数据，跳过标题行
    
    # 使用字典来整合重复的国家数据
    data_dict = {}
    for line in lines:
        values = line.strip().split(',')
        country = values[0]
        
        # 如果国家不在字典中，将其添加到字典中
        if country not in data_dict:
            data_dict[country] = [float(i) for i in values[1:]]
        else:
            # 如果国家已经在字典中，累加数据
            for i in range(len(data_dict[country])):
                data_dict[country][i] += float(values[i + 1])

# 创建一个新的矩阵
new_matrix = [header]
for country, data in data_dict.items():
    new_matrix.append([country] + data)

# 将新的矩阵保存为CSV
output_file_path = "G:\\Hangkai\\Forest_edge\\country_stats\\final_data\\aggregated_forest_edge_country.csv"
with open(output_file_path, 'w', encoding='ISO-8859-1') as f:
    for row in new_matrix:
        f.write(','.join(map(str, row)) + '\n')

print(f"Aggregated data saved to {output_file_path}")

Aggregated data saved to G:\Hangkai\Forest_edge\country_stats\final_data\aggregated_forest_edge_country.csv


In [5]:
file_path = "G:\\Hangkai\\Forest_edge\\country_stats\\final_data\\aggregated_forest_edge_country.csv"

new_matrix = []

with open(file_path, 'r', encoding='ISO-8859-1') as f:
    lines = f.readlines()[1:]  # 跳过第一行 (header)
    
    # 添加新的矩阵的header
    new_matrix.append(['country', 'stable', 'stable_per2000','stable_per2020',
                       'increase increase','increase increase percent',
                       'increase decrease', 'increase decrease percent',
                       'decrease increase', 'decrease increase percent',
                       'decrease decrease', 'decrease decrease percent',
                       'total increase per','total increase',
                       'total decrease per','total decrease',
                       'forest edge 2000', 'forest edge 2020'])
    
    for line in lines:
        values = line.strip().split(',')
        country = values[0]
        stable = float(values[1]) + float(values[8]) + float(values[15])
        increase_increase = float(values[7]) - float(values[6])
        increase_decrease = float(values[9]) - float(values[8])
        decrease_increase = float(values[14]) - float(values[15])
        decrease_decrease = float(values[16]) - float(values[17])
        increase = increase_increase + increase_decrease
        decrease = decrease_increase + decrease_decrease

        forest_edge_2000 = stable + decrease_increase + decrease_decrease
        forest_edge_2020 = stable + increase_increase + increase_decrease
        
        if stable == 0:
            stable_per2000 = -1
        else:
            stable_per2000 = stable/forest_edge_2000

        if stable == 0:
            stable_per2020 = -1
        else:
            stable_per2020 = stable/forest_edge_2020
        
        if forest_edge_2020 == 0:
            increase_increase = -1
            increase_decrease = -1
            decrease_increase = -1
            decrease_decrease = -1
            increase_decrease_per = -1
            increase_increase_per = -1
            decrease_increase_per = -1
            decrease_decrease_per = -1
            increase_per = -1
            decrease_per = -1
            increase = -1
            decrease = -1
            stable = -1
            forest_edge_2000 = -1
            forest_edge_2020 = -1
        else:
            increase_decrease_per = increase_decrease/forest_edge_2020
            increase_increase_per = increase_increase/forest_edge_2020
            decrease_increase_per = decrease_increase/forest_edge_2000
            decrease_decrease_per = decrease_decrease/forest_edge_2000

            increase_per = increase_increase_per+increase_decrease_per
            decrease_per = decrease_increase_per+decrease_decrease_per
        new_matrix.append([country, stable,stable_per2000,stable_per2020,
                           increase_increase,increase_increase_per,
                           increase_decrease,increase_decrease_per,
                           decrease_increase,decrease_increase_per,
                           decrease_decrease,decrease_decrease_per,
                           increase_per,increase,decrease_per,decrease,
                           forest_edge_2000, forest_edge_2020])

# 将新的矩阵保存为CSV
output_file_path = "G:\\Hangkai\\Forest_edge\\country_stats\\final_data\\processed_country_data.csv"
with open(output_file_path, 'w', encoding='ISO-8859-1') as f:
    for row in new_matrix:
        f.write(','.join(map(str, row)) + '\n')

print(f"Processed data saved to {output_file_path}")



Processed data saved to G:\Hangkai\Forest_edge\country_stats\final_data\processed_country_data.csv


In [3]:
import cartopy.io.shapereader as shpreader

# 读取 CSV 文件
file_path = "G:\\Hangkai\\Forest_edge\\country_stats\\final_data\\processed_country_data.csv"

with open(file_path, 'r', encoding='ISO-8859-1') as f:
    lines = f.readlines()
    header = lines[0].strip().split(',')
    lines = lines[1:]  # 仅获取数据，跳过标题行
    
    # 使用字典来存储国家数据
    data_dict = {}
    for line in lines:
        values = line.strip().split(',')
        country = values[0]
        data_dict[country] = [float(i) for i in values[1:]]

# 1. 从 natural earth shape file 中获取国家到大洲的映射
reader = shpreader.Reader(shpreader.natural_earth(resolution='10m', category='cultural', name='admin_0_countries'))
country_continent_mapping = {record.attributes['SOVEREIGNT']: record.attributes['CONTINENT'] for record in reader.records()}

# 2. 使用上面的映射按大洲汇总国家数据
continent_data_dict = {}

for country, data in data_dict.items():
    # 获取国家对应的大洲
    continent = country_continent_mapping.get(country, None)
    if continent:
        if continent not in continent_data_dict:
            continent_data_dict[continent] = [0] * len(data)
        
        for i in range(len(data)):
            continent_data_dict[continent][i] += data[i]

# 3. 保存每个大洲的汇总数据到一个新的CSV文件
output_file_path = "G:\\Hangkai\\Global forest edge\\country_forest_area_output\\aggregated_forest_edge_by_continent.csv"
with open(output_file_path, 'w', encoding='ISO-8859-1') as f:
    f.write(','.join(header) + '\n')
    for continent, data in continent_data_dict.items():
        f.write(','.join([continent] + list(map(str, data))) + '\n')

print(f"Aggregated data by continent saved to {output_file_path}")


Aggregated data by continent saved to G:\Hangkai\Global forest edge\country_forest_area_output\aggregated_forest_edge_by_continent_by_continent.csv
